In [ ]:
# Imports and MySQL connection

import mysql.connector
import json
import requests
import time
import csv
import config
api_key = config.api_key
config.host
cnx = mysql.connector.connect(
                              host = config.host,
                              user = config.user,
                              passwd = config.passwd,
                              database = "Movie Project1"
                              )
cursor = cnx.cursor()
print(cnx)

In [ ]:
# Got the movie titles for each genre from the csv files

with open('Romcom_Movies.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    movie_titles = []
    for movie in reader:
        movie_titles.append(movie['title'])

In [ ]:
movie_titles

In [ ]:
len(movie_titles)

In [ ]:
# API call function

def all_results(api_key, movies):
    results = []
    for i in range(len(movies)):
        title = movies[i]
        url_params = {'term': title.replace(' ', '+')}
        url = f'http://www.omdbapi.com/?apikey={api_key}&t={title}'
        response = requests.get(url, params = url_params)
        results.append(response.json())
        time.sleep(1) # wait a second
    return results

In [ ]:
test = all_results(api_key, movie_titles)

In [ ]:
test

In [ ]:
# Extracted just the IMDb ratings and inserted into the designated table

def all_ratings(movies):
    all_data = []
    for movie in movies:
        if 'Title' in movie.keys():
            try:
                data = {'Title': movie['Title'],
                       'imdbRating': float(movie['imdbRating'])}
            except:
                continue
            cursor = cnx.cursor()
            try:
                cursor.execute("""INSERT INTO Reviews (title, ratings)
                                           VALUES (%s, %s)""", (data['Title'], data['imdbRating']))
            except:
                pass
            cnx.commit()
            all_data.append(data)
    return all_data

In [ ]:
final_test = all_ratings(test)

In [ ]:
final_test

In [ ]:
len(final_test)

In [ ]:
# Saved the ratings for each genre in a separate csv file

data_list = final_test
keys = data_list[0].keys()
with open('Romcom_Ratings.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(data_list)